## **Ejercicio**:

Entrenar una red neuronal MLP para el problema MNIST. Los valores que se quieren probar son los siguientes:

-   Número de capas ocultas [1,2,3]
-   Dimensión de las capas ocultas [512,1024]
-   Learning rate [0.01, 0.001]
-   Batch_size [128]
-   Epochs con early_stopping

Obtener la mejor combinación con el conjunto de validación y probarla finalmente sobre el conjunto de test

In [ ]:
vlayers=[1,2,3]
vdim=[512,1024]
vlr=[0.025,0.01]
batch_size=128

best=0.0
for layers in vlayers:
    for dim in vdim:
        for lr in vlr:
            print("===========================")
            print("Num hidden layers=",layers)
            print("Dim hidden layers=",dim)
            print("Learning rate=",lr)
            print("===========================")

            model = Sequential()

            model.add(Input(784))
            for i in range(layers):
                model.add(Dense(dim, activation='relu'))
            model.add(Dense(num_classes, activation='softmax'))

            sgd=SGD(learning_rate=lr, momentum=0.9)

            callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación

            model.compile(loss='categorical_crossentropy',
                          optimizer=sgd,
                          metrics=['accuracy'])

            epochs=100  ## No nos preocupemos, el fit acabará antes por el early_stopping
            history = model.fit(x_train, y_train,
                                batch_size=batch_size,
                                epochs=epochs,
                                verbose=1,
                                validation_data=(x_val, y_val),
                                callbacks=[callback])
            res=max(history.history['val_accuracy'][:]) ## <--- el mejor, no el último, luego aprenderemos a guardarnos este mejor y no quedarnos con el último modelo
            print("\n RES=",res,"\n")
            if res>best:
                best=res
                print("***********************")
                best_dim=dim
                best_layers=layers
                best_lr=lr

print("===========================")
print("Best num hidden layers=",best_layers)
print("Best dim hidden layers=",best_dim)
print("Best learning rate=",best_lr)
print("===========================")

## PROBAR EL MEJOR CON TEST
model = Sequential()

model.add(Input(784))
for i in range(best_layers):
    model.add(Dense(best_dim, activation='relu'))  
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=best_lr, momentum=0.9)


callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

epochs=100  ## No nos preocupemos, el fit acabará antes por el early_stopping
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[callback])



## Ajustar parámetros

Es posible que en el ejemplo anterior el entrenamiento haya acabado prematuramente por el efecto del Early Stopping. En ese caso, el modelo que se ha obtenido no es el mejor posible. Para obtener el mejor modelo quizás sea necesario aumentar el número de epochs y para ello modificar algún parámetro del early stopping.

## Ejercicio: 

Modificar el early stopping y emplear un modelo con los mejores parámetros (número de capas ocultas, learning rate, batch size, etc) para evaluar finalmente el modelo guardado sobre el test.


In [ ]:
## Solución 

# Cada uno que pruebe con la mejor topología, en mi caso 3 capas ocultas de 1024 neuronas cada una 
# y learning rate 0.025
# Pero lo importante es que reduzcan el min_delta a 0.001 o incluso menos, en mi caso 0.0001

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.025, momentum=0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    
 

callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3) 

# Añade un model checkpoint para guardar el mejor modelo
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True,verbose=1)

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[callback, checkpoint])s

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
